<a href="https://colab.research.google.com/github/sanj1210/PALM/blob/main/aqua_bison_instruction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install google-cloud-aiplatform==1.36.2 --upgrade --user
!pip uninstall bigframes
!pip install bigframes==0.26.0


import bigframes.dataframe
import IPython
import time  # for delays

from datasets import load_dataset, concatenate_datasets

app = IPython.Application.instance()
app.kernel.do_shutdown(True)


import vertexai


PROJECT_ID = "able-scope-436106-s0"  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}


# Initialize Vertex AI SDK
vertexai.init(project=PROJECT_ID, location=REGION)

import sys

if "google.colab" in sys.modules:
   from google.colab import auth
   auth.authenticate_user()

from datasets import load_dataset
import re  # this is for searching

import pandas as pd
import seaborn as sns
from IPython.display import Markdown, display
from sklearn.metrics.pairwise import cosine_similarity
from vertexai.language_models import (
   TextGenerationModel,
   TextEmbeddingModel,
   ChatModel,
   InputOutputTextPair,
   CodeGenerationModel,
   CodeChatModel,
)

# Load the AQUA dataset
dataset = load_dataset("aqua_rat")
aqua_eval = pd.DataFrame(data=dataset["validation"])
print(dataset["validation"][0])  # Print a sample of the validation set

# Prepare the full question by merging question text with answer choices
questions_full = []
for ind in aqua_eval.index:
    options_merged = (" ".join(aqua_eval["options"][ind])).lower()  # Join all answer options
    quest = aqua_eval["question"][ind] + "\nAnswer Choices: " + options_merged  # Format question + options
    questions_full.append(quest)

aqua_eval["Full_Question"] = questions_full
aqua_eval["Int_Answer"] = aqua_eval["correct"]  # Extract correct answers for comparison


# Print the number of questions in the dataset
num_questions = len(aqua_eval)
print(f"Number of questions in the dataset: {num_questions}")

# Process only the first fifth of the dataset
start_index = 0 * num_questions
end_index = num_questions
subset_aqua_eval = aqua_eval.iloc[start_index:end_index]

# Exemplars for model training
exemplars = '''
Think step by step to answer the given question. The last sentence of the answer must be of the format `The answer is (x)`.
Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The answer is 39.
Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
A: Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8. The answer is 8.
Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?
A: Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9. The answer is 9.
Q: There were nine computers in the server room. Five more computers were installed each day, from Monday to Thursday. How many computers are now in the server room?
A: There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29. The answer is 29.
Q: Michael had 58 golf balls. On Tuesday, he lost 23 golf balls. On Wednesday, he lost 2 more. How many golf balls did he have at the end of Wednesday?
A: Michael started with 58 golf balls. After losing 23 on Tuesday, he had 58 - 23 = 35. After losing 2 more, he had 35 - 2 = 33 golf balls. The answer is 33.
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: Olivia had 23 dollars. 5 bagels for 3 dollars each will be 5 x 3 = 15 dollars. So she has 23 - 15 dollars left. 23 - 15 is 8. The answer is 8.
'''

# Instruction for the AQUA dataset
instruction = '''
Think step by step to answer the given question.
'''

# Initialize the model
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

# Updated regex pattern and extraction method
pattern = r'\banswer\s+is\s+(\(?[A-Za-z]+\)?)[\s\S]*\.*'

# Function to extract the answer using the new regex
def extract_answer_aqua_regex(output):
    try:
        match = re.search(pattern, output.lower())
        if match:
            palm_answer = match.group(1)
            # Process the extracted answer to account for '(a)' or 'a)'
            if palm_answer[0] == '(' and palm_answer[-1] == ')':
                palm_opt = palm_answer[1]
            elif palm_answer[-1] == ')':
                palm_opt = palm_answer[0]
            else:
                palm_opt = palm_answer  # In case it's a single letter without parentheses
            return palm_opt.lower()
        return "No answer"
    except Exception as e:
        print(f"Error extracting answer with regex: {e}")
        return "No answer"

# Function to process a batch of questions
def process_batch_aqua(batch):
    responses = []
    for question in batch:
        input_prompt = instruction + "\n" + exemplars + "\n" + "Q: " + question + "\n" + "A: "
        try:
            response = generation_model.predict(prompt=input_prompt, max_output_tokens=1024, temperature=0.2, top_k=0.8, top_p=0.4)
            responses.append(response.text)
        except Exception as e:
            print("Exception caught:", e)
            responses.append("Error: Quota Exceeded")
        time.sleep(2)  # Adding a delay between requests
    return responses

# Process the subset of the dataset in batches
batch_size = 5
results = []
questions_to_process = subset_aqua_eval['Full_Question']

for i in range(0, len(questions_to_process), batch_size):
    batch_questions = questions_to_process[i:i + batch_size]
    batch_responses = process_batch_aqua(batch_questions)
    results.extend(batch_responses)
    print(f"Processed batch {i // batch_size + 1}")
    time.sleep(47)

# Extract ground truth answers and model responses
ids = list(range(start_index, start_index + len(subset_aqua_eval)))
ground_truth_answers = subset_aqua_eval['Int_Answer']

# Update the extraction method here
extracted_values = [extract_answer_aqua_regex(response) for response in results]
ground_truth_values = [ans.lower() for ans in ground_truth_answers]  # Ensure both are lowercase

# Function to compare extracted and ground truth values
def compare_values_aqua(extracted, ground_truth):
    return extracted == ground_truth

# Calculate correctness
correctness = [1 if compare_values_aqua(extracted, ground_truth) else 0 for extracted, ground_truth in zip(extracted_values, ground_truth_values)]

# Create a dataframe to store the results
results_df = pd.DataFrame({
    'id': ids,
    'question': subset_aqua_eval['Full_Question'],
    'groundTruthAns': ground_truth_answers,
    'model output': results,
    'extracted value': extracted_values,
    'ground truth value': ground_truth_values,
    'correctness': correctness
})
from google.colab import files

# Save the results to a csv file
results_df.to_csv("responses_aqua_subset.csv", index=False)
files.download("responses_aqua_subset.csv")

# Function to compute accuracy and save results
def compute_accuracy_and_save_aqua(results_df, dataset_name, model_name, prompt_type):
    accuracy = results_df['correctness'].mean()
    output_content = f"{{Dataset: {dataset_name}, \nmodel_name: {model_name}, \naccuracy: {accuracy}, \nprompt: {prompt_type}}}"
    with open(f"accuracy_aqua_subset.txt", "w") as f:
        f.write(output_content)
    print(output_content)

# Compute accuracy and save results
compute_accuracy_and_save_aqua(results_df, "AQUA", "text-bison@001", "COT with instruction")
files.download("accuracy_aqua_subset.txt")




Found existing installation: bigframes 1.17.0
Uninstalling bigframes-1.17.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/bigframes-1.17.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/bigframes/*
    /usr/local/lib/python3.10/dist-packages/bigframes_vendored/*
Proceed (Y/n)? Y
  Successfully uninstalled bigframes-1.17.0
  Using cached bigframes-0.26.0-py2.py3-none-any.whl.metadata (22 kB)
  Using cached pandas-2.1.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached pyarrow-15.0.2-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (3.0 kB)
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached bigframes-0.26.0-py2.py3-none-any.whl (457 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 19.2 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  Attempting uninstall: pyarrow
    Found existing ins

README.md:   0%|          | 0.00/5.89k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/25.4M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/74.0k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/76.1k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/97467 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/254 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/254 [00:00<?, ? examples/s]

{'question': 'Three birds are flying at a fast rate of 900 kilometers per hour. What is their speed in miles per minute? [1km = 0.6 miles]', 'options': ['A)32400', 'B)6000', 'C)600', 'D)60000', 'E)10'], 'rationale': 'To calculate the equivalent of miles in a kilometer\n0.6 kilometers = 1 mile\n900 kilometers = (0.6)*900 = 540 miles\nIn 1 hour there are 60 minutes\nSpeed in miles/minutes = 60 * 540 = 32400\nCorrect answer - A', 'correct': 'A'}
Number of questions in the dataset: 254
Processed batch 1
Processed batch 2
Processed batch 3
